In [22]:
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns

from pathlib import Path
from tqdm.autonotebook import tqdm
from sklearn.model_selection import train_test_split         # Split data into train and test set

from utils import evaluate_classifier, get_sklearn_df 

# Automatically prints execution time for the individual cells
%load_ext autotime

# Automatically reloads functions defined in external files
%load_ext autoreload
%autoreload 2

# Set xarray to use html as display_style
xr.set_options(display_style="html")

# Tell matplotlib to plot directly in the notebook
%matplotlib inline  

# The path to the project (so absoute file paths can be used throughout the notebook)
PROJ_PATH = Path.cwd().parent

# Set seed for random generators
RANDOM_SEED = 42

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 26.9 ms


In [2]:
netcdf_path = (PROJ_PATH / 'data' / 'processed' / 'FieldPolygons2019_stats').with_suffix('.nc')
ds = xr.open_dataset(netcdf_path, engine="h5netcdf")
ds  # Remember to close the dataset before the netcdf file can be rewritten in cells above

<xarray.Dataset>
Dimensions:         (date: 77, field_id: 67312, polarization: 3)
Coordinates:
  * date            (date) datetime64[ns] 2018-07-08 2018-07-14 ... 2019-10-31
  * field_id        (field_id) int32 61853445 61952339 ... 62131272 63199619
  * polarization    (polarization) object 'VH' 'VV' 'VV-VH'
Data variables:
    afgkode         (field_id) int16 ...
    afgroede        (field_id) object ...
    cvr             (field_id) object ...
    gb              (field_id) float32 ...
    gbanmeldt       (field_id) float32 ...
    imk_areal       (field_id) float32 ...
    journalnr       (field_id) object ...
    marknr          (field_id) object ...
    pass_mode       (date) object ...
    relative_orbit  (date) int16 ...
    satellite       (date) object ...
    stats_max       (field_id, date, polarization) float32 ...
    stats_mean      (field_id, date, polarization) float32 ...
    stats_median    (field_id, date, polarization) float32 ...
    stats_min       (field_id, date, polarization) float32 ...
    stats_std       (field_id, date, polarization) float32 ...

time: 2.27 s


In [3]:
ds.close()

time: 158 ms


In [4]:
# Convert the xarray dataset to pandas dataframe
df = ds.to_dataframe()
df = df.reset_index()  # Removes MultiIndex
df = df.drop(columns=['cvr', 'gb', 'gbanmeldt', 'journalnr', 'marknr', 'pass_mode', 'relative_orbit'])
df = df.dropna()

time: 2min 56s


In [27]:
year = 2018
for i in range(7, 24, 1):
    month = (i % 12) + 1
    if month == 1:
        year += 1
        
    print(f"--------------------------------------------------------------------------------------------------")
    print(f"Dataset from 2018-07-01 to {year}-{month:02}-01")
    df_sklearn = get_sklearn_df(polygons_year=2019, 
                                satellite_dates=slice('2018-07-01', f'{year}-{month:02}-01'), 
                                fields='all', 
                                satellite='all', 
                                polarization='all',
                                crop_type='all',
                                netcdf_path=netcdf_path)

    df_sklearn = df_sklearn[df_sklearn['afgroede'].isin(['Vårbyg', 'Vinterhvede', 'Silomajs', 'Vinterraps', 
                                                         'Vinterbyg', 'Vårhavre', 'Vinterhybridrug'])]
    crop_codes = df_sklearn['afgkode'].unique()
    mapping_dict = {}
    class_names = [] 

    for i, crop_code in enumerate(crop_codes):
        mapping_dict[crop_code] = i
        crop_type = df_sklearn[df_sklearn['afgkode'] == crop_code].head(1)['afgroede'].values[0]
        class_names.append(crop_type)

    df_sklearn_remapped = df_sklearn.copy()
    df_sklearn_remapped['afgkode'] = df_sklearn_remapped['afgkode'].map(mapping_dict)
    #print(f"Crop types: {class_names}")

    array = df_sklearn_remapped.values

    # Define the independent variables as features.
    X = np.float32(array[:,3:])  # The features 

    # Define the target (dependent) variable as labels.
    y = np.int8(array[:,1])  # The column 'afgkode'

    # Create a train/test split using 30% test size.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_SEED)

    #print(f"Train samples:      {len(y_train)}")
    #print(f"Test samples:       {len(y_test)}")
    #print(f"Number of features: {len(X[0,:])}")

    from sklearn.linear_model import LogisticRegression          

    # Instantiate and evaluate classifier
    clf = LogisticRegression(solver='lbfgs', multi_class='auto', n_jobs=10, max_iter=1000)
    clf_trained = evaluate_classifier(clf, X_train, X_test, y_train, y_test, class_names, feature_scale=True, plot_confusion_matrix=False)

--------------------------------------------------------------------------------------------------
Dataset from 2018-07-01 to 2018-08-01
Training time: 0.635 s
Prediction time: 0.001 s

Report:

Train accuracy: 0.3548
Test accuracy: 0.3554

                  precision    recall  f1-score   support

         Vårbyg       0.23      0.07      0.11      1192
    Vinterhvede       0.36      0.20      0.25      1092
       Silomajs       0.48      0.67      0.56      1469
     Vinterraps       0.36      0.32      0.34      1125
      Vinterbyg       0.25      0.38      0.30      1293
       Vårhavre       0.28      0.30      0.29      1312
Vinterhybridrug       0.43      0.45      0.44      1412

       accuracy                           0.36      8895
      macro avg       0.34      0.34      0.33      8895
   weighted avg       0.34      0.36      0.34      8895

--------------------------------------------------------------------------------------------------
Dataset from 2018-07-01 to 20

In [ ]:
year = 2018
for i in range(7, 24, 1):
    month = (i % 12) + 1
    if month == 1:
        year += 1
        
    print(f"--------------------------------------------------------------------------------------------------")
    print(f"Dataset from 2018-07-01 to {year}-{month:02}-01")
    df_sklearn = get_sklearn_df(polygons_year=2019, 
                                satellite_dates=slice('2018-07-01', f'{year}-{month:02}-01'), 
                                fields='all', 
                                satellite='all', 
                                polarization='all',
                                crop_type='all',
                                netcdf_path=netcdf_path)

    df_sklearn = df_sklearn[df_sklearn['afgroede'].isin(['Vårbyg', 'Vinterhvede', 'Silomajs', 'Vinterraps', 
                                                         'Vinterbyg', 'Vårhavre', 'Vinterhybridrug'])]
    crop_codes = df_sklearn['afgkode'].unique()
    mapping_dict = {}
    class_names = [] 

    for i, crop_code in enumerate(crop_codes):
        mapping_dict[crop_code] = i
        crop_type = df_sklearn[df_sklearn['afgkode'] == crop_code].head(1)['afgroede'].values[0]
        class_names.append(crop_type)

    df_sklearn_remapped = df_sklearn.copy()
    df_sklearn_remapped['afgkode'] = df_sklearn_remapped['afgkode'].map(mapping_dict)
    #print(f"Crop types: {class_names}")

    array = df_sklearn_remapped.values

    # Define the independent variables as features.
    X = np.float32(array[:,3:])  # The features 

    # Define the target (dependent) variable as labels.
    y = np.int8(array[:,1])  # The column 'afgkode'

    # Create a train/test split using 30% test size.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    #print(f"Train samples:      {len(y_train)}")
    #print(f"Test samples:       {len(y_test)}")
    #print(f"Number of features: {len(X[0,:])}")

    from sklearn.linear_model import LogisticRegressionCV          

    # Instantiate and evaluate classifier
    clf = LogisticRegressionCV(solver='lbfgs', multi_class='auto', cv=10, n_jobs=10, random_state=RANDOM_SEED, max_iter=1000)
    clf_trained = evaluate_classifier(clf, X_train, X_test, y_train, y_test, class_names, feature_scale=True, plot_confusion_matrix=False)

--------------------------------------------------------------------------------------------------
Dataset from 2018-07-01 to 2018-08-01
Training time: 11.2718 s
Prediction time: 0.0016 s

Report:

Train accuracy: 0.3525
Test accuracy: 0.3559

                  precision    recall  f1-score   support

         Vårbyg       0.19      0.08      0.11      1149
    Vinterhvede       0.38      0.20      0.26      1122
       Silomajs       0.48      0.70      0.57      1432
     Vinterraps       0.35      0.32      0.34      1095
      Vinterbyg       0.26      0.39      0.31      1306
       Vårhavre       0.28      0.27      0.28      1376
Vinterhybridrug       0.42      0.44      0.43      1415

       accuracy                           0.36      8895
      macro avg       0.34      0.34      0.33      8895
   weighted avg       0.34      0.36      0.34      8895

--------------------------------------------------------------------------------------------------
Dataset from 2018-07-01 to

In [ ]:
year = 2018
for i in range(7, 24, 1):
    month = (i % 12) + 1
    if month == 1:
        year += 1
        
    print(f"--------------------------------------------------------------------------------------------------")
    print(f"Dataset from 2018-07-01 to {year}-{month:02}-01")
    df_sklearn = get_sklearn_df(polygons_year=2019, 
                                satellite_dates=slice('2018-07-01', f'{year}-{month:02}-01'), 
                                fields='all', 
                                satellite='all', 
                                polarization='all',
                                crop_type='all',
                                netcdf_path=netcdf_path)

    df_sklearn = df_sklearn[df_sklearn['afgroede'].isin(['Vårbyg', 'Vinterhvede', 'Silomajs', 'Vinterraps', 
                                                         'Vinterbyg', 'Vårhavre', 'Vinterhybridrug'])]
    crop_codes = df_sklearn['afgkode'].unique()
    mapping_dict = {}
    class_names = [] 

    for i, crop_code in enumerate(crop_codes):
        mapping_dict[crop_code] = i
        crop_type = df_sklearn[df_sklearn['afgkode'] == crop_code].head(1)['afgroede'].values[0]
        class_names.append(crop_type)

    df_sklearn_remapped = df_sklearn.copy()
    df_sklearn_remapped['afgkode'] = df_sklearn_remapped['afgkode'].map(mapping_dict)
    #print(f"Crop types: {class_names}")

    array = df_sklearn_remapped.values

    # Define the independent variables as features.
    X = np.float32(array[:,3:])  # The features 

    # Define the target (dependent) variable as labels.
    y = np.int8(array[:,1])  # The column 'afgkode'

    # Create a train/test split using 30% test size.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    #print(f"Train samples:      {len(y_train)}")
    #print(f"Test samples:       {len(y_test)}")
    #print(f"Number of features: {len(X[0,:])}")

    from sklearn.linear_model import LogisticRegressionCV          

    # Instantiate and evaluate classifier
    clf = LogisticRegressionCV(solver='lbfgs', multi_class='auto', n_jobs=10, cv=10, random_state=RANDOM_SEED, max_iter=1000, class_weight='balanced')
    clf_trained = evaluate_classifier(clf, X_train, X_test, y_train, y_test, class_names, feature_scale=True, plot_confusion_matrix=False)

In [ ]:
year = 2018
for i in range(7, 24, 1):
    month = (i % 12) + 1
    if month == 1:
        year += 1
        
    print(f"--------------------------------------------------------------------------------------------------")
    print(f"Dataset from 2018-07-01 to {year}-{month:02}-01")
    df_sklearn = get_sklearn_df(polygons_year=2019, 
                                satellite_dates=slice('2018-07-01', f'{year}-{month:02}-01'), 
                                fields='all', 
                                satellite='all', 
                                polarization='all',
                                crop_type='all',
                                netcdf_path=netcdf_path)

    df_sklearn = df_sklearn[df_sklearn['afgroede'].isin(['Vårbyg', 'Vinterhvede', 'Silomajs', 'Vinterraps', 
                                                         'Vinterbyg', 'Vårhavre', 'Vinterhybridrug'])]
    crop_codes = df_sklearn['afgkode'].unique()
    mapping_dict = {}
    class_names = [] 

    for i, crop_code in enumerate(crop_codes):
        mapping_dict[crop_code] = i
        crop_type = df_sklearn[df_sklearn['afgkode'] == crop_code].head(1)['afgroede'].values[0]
        class_names.append(crop_type)

    df_sklearn_remapped = df_sklearn.copy()
    df_sklearn_remapped['afgkode'] = df_sklearn_remapped['afgkode'].map(mapping_dict)
    #print(f"Crop types: {class_names}")

    array = df_sklearn_remapped.values

    # Define the independent variables as features.
    X = np.float32(array[:,3:])  # The features 

    # Define the target (dependent) variable as labels.
    y = np.int8(array[:,1])  # The column 'afgkode'

    # Create a train/test split using 30% test size.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_SEED)

    #print(f"Train samples:      {len(y_train)}")
    #print(f"Test samples:       {len(y_test)}")
    #print(f"Number of features: {len(X[0,:])}")

    from sklearn.svm import SVC   

    # Instantiate and evaluate classifier
    clf = SVC(kernel='rbf')
    clf_trained = evaluate_classifier(clf, X_train, X_test, y_train, y_test, class_names, feature_scale=True, plot_confusion_matrix=False)